In [1]:
import cv2
import numpy as np
import pytesseract
import grip
import time

cap = cv2.VideoCapture("/Users/akshar/Downloads/Humans in Autonomy/DroneFootageAnalysis/SampleVid.mp4")
REFERENCE_INSPECTION_FRAME = cv2.imread("/Users/akshar/Downloads/Humans in Autonomy/DroneFootageAnalysis/blankFrame.jpg")
CROPPED_REFERENCE = cv2.imread("/Users/akshar/Downloads/Humans in Autonomy/DroneFootageAnalysis/RefenceUnderJoystick.jpg")
INSPECTION_FRAME_SIMILARITY_THRESHOLD = 60

cv2.startWindowThread()

def isInspectionMode(frame):
    difference = cv2.absdiff(REFERENCE_INSPECTION_FRAME, frame)
    res = difference.astype(np.uint8)
    percentage = abs(100 - ((np.count_nonzero(res) * 100)/ res.size))

    if percentage >= INSPECTION_FRAME_SIMILARITY_THRESHOLD:
        return True
    else:
        return False
    
def isInspectionClick(frame):
    cropped_frame_left = frame[550:650, 20:300]
    cropped_frame_right = frame[550:650, 990:1270]
    if isInspectionMode(cropped_frame):
        time_frame = frame[10:75, 600:700]
        time_str = pytesseract.image_to_string(time_frame, config='--psm 6')

        difference = cv2.absdiff(CROPPED_REFERENCE, cropped_frame)
        res = difference.astype(np.uint8)
        percentage = abs(100 - ((np.count_nonzero(res) * 100)/ res.size))

        if percentage < 100:
            return True
        else:
            return False
    else:
        print ("Not an Inspection Frame")
        return False
    
def flightTime(frame):
    time_frame = frame[10:75, 600:700]
    time_str = pytesseract.image_to_string(time_frame, config='--psm 6')
    try:
        seconds = (time.strptime(time_str, '%M:%S')[4]*60) + time.strptime(time_str, '%M:%S')[5]
        return seconds
    except:
        return
    
def checkDifference(frame):
    print (frame.shape, CROPPED_REFERENCE.shape)
    difference = cv2.subtract(frame, CROPPED_REFERENCE)
    low_threshold = 50
    high_threshold = 150
    edges = cv2.Canny(difference, low_threshold, high_threshold)
    rho = 1  # distance resolution in pixels of the Hough grid
    theta = np.pi / 180  # angular resolution in radians of the Hough grid
    threshold = 15  # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 200  # minimum number of pixels making up a line
    max_line_gap = 200  # maximum gap in pixels between connectable line segments
    line_image = np.copy(frame) * 0  # creating a blank to draw lines on

    # Run Hough on edge detected image
    # Output "lines" is an array containing endpoints of detected line segments
    lines = cv2.HoughLinesP(edges, rho, theta, threshold, np.array([]),
                        min_line_length, max_line_gap)
    
    if lines:
        if len(lines) > 0:
            for line in lines:
                for x1,y1,x2,y2 in line:
                    print(x1, y1, x2, x2)
                    cv2.line(line_image,(x1,y1),(x2,y2),(255,0,0),5)
        # Draw the lines on the  image
        return cv2.addWeighted(frame, 0.8, line_image, 1, 0)
    
    

ModuleNotFoundError: No module named 'cv2'

In [2]:
count = 0

if cap.isOpened(): # Check if camera opened successfully
    while cap.isOpened():
        ret, frame = cap.read()
        if ret == True:
            difference = cv2.absdiff(REFERENCE_INSPECTION_FRAME, frame)
            res = difference.astype(np.uint8)
            percentage = abs(100 - ((np.count_nonzero(res) * 100)/ res.size))
            
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, str(percentage),(100,100), cv2.FONT_HERSHEY_SIMPLEX, 1.0,(255,0,255),lineType=cv2.LINE_AA)
            # cv2.imshow("Video", frame)
            
            if percentage >= INSPECTION_FRAME_SIMILARITY_THRESHOLD:
                
            
            
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
        else:
            break
            
else:
    print("Error opening video stream or file")

IndentationError: expected an indented block (<ipython-input-2-98503db61486>, line 19)

In [3]:
while cap.isOpened():
    ret, frame = cap.read()
    if ret == True:
        cropped_frame = frame[550:650, 0:1280]
        cv2.imshow("cropped frame", cropped_frame)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

NameError: name 'cap' is not defined

In [ ]:
frame_width = int( CROPPED_REFERENCE.shape[0])
frame_height =int( CROPPED_REFERENCE.shape[1])
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('/Users/akshar/Downloads/Humans in Autonomy/DroneFootageAnalysis/InspectionMode.avi', fourcc, 1, (frame_height, frame_width))
while cap.isOpened():
    ret, frame = cap.read()
    if ret == True:
        cropped_frame = frame[550:650, 0:1280]
        if isInspectionMode(cropped_frame):
            out.write(cropped_frame)
            print ("Added frame to video.")
        else:
            print ("Skipped frame")
    else:
        break
print ("Done.")

In [ ]:
count = 0
# Extracted Seconds from App
while cap.isOpened():
    ret, frame = cap.read()
    if ret == True:
        cropped_frame = frame[550:650, 0:1280]
        if isInspectionMode(cropped_frame):
            time_frame = frame[10:75, 600:700]
            time_str = pytesseract.image_to_string(time_frame, config='--psm 6')
            
            difference = cv2.absdiff(CROPPED_REFERENCE, cropped_frame)
            res = difference.astype(np.uint8)
            percentage = abs(100 - ((np.count_nonzero(res) * 100)/ res.size))
            
            cv2.putText(frame, "%s - percentage. %s - time" % (percentage, time_str), (0, 0), cv2.FONT_HERSHEY_SIMPLEX, 1.0,(255,0,255),lineType=cv2.LINE_AA)
            # cv2.imshow("time", time_frame)
            # cv2.imshow("frame", frame)
            if percentage < 100:
                print ("Writing frame %d to disk with percent similarity of %d" % (count, percentage))
                cv2.imwrite("/Users/akshar/Downloads/Humans in Autonomy/DroneFootageAnalysis/frame%d.jpg" % count, frame)
                count += 1
            print (percentage, count)
        
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
print("Done.")

In [ ]:
count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if ret == True:
        if isInspectionClick(frame):
            cv2.imwrite("/Users/akshar/Downloads/Humans in Autonomy/DroneFootageAnalysis/InspectionClickFrame%d.jpg" % count, frame)
            print (flightTime(frame))
            count += 1

In [18]:
print ("Running")
count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if ret == True:
        cropped_frame_left = frame[550:650, 20:300]
        cropped_frame_right = frame[550:650, 990:1270]
        time_frame = frame[10:75, 600:700]
        if isInspectionMode(frame):
            try:
                cv2.imwrite("/Users/akshar/Downloads/Humans in Autonomy/DroneFootageAnalysis/Lines%d.jpg" % (count), checkDifference(cropped_frame_left))
            except:
                print("Did not write image, no lines were found")
            print (count, flightTime(frame))
            count += 1
        if cv2.waitKey(25) & 0xFF == ord('q'):
            cv2.imwrite("/Users/akshar/Downloads/Humans in Autonomy/DroneFootageAnalysis/RefenceUnderJoystick.jpg", cropped_frame)

Running
(100, 280, 3) (100, 280, 3)
0 None
(100, 280, 3) (100, 280, 3)
1 None
(100, 280, 3) (100, 280, 3)
2 None
(100, 280, 3) (100, 280, 3)
3 None
(100, 280, 3) (100, 280, 3)
4 None
(100, 280, 3) (100, 280, 3)
5 None
(100, 280, 3) (100, 280, 3)
6 None
(100, 280, 3) (100, 280, 3)
7 None
(100, 280, 3) (100, 280, 3)
8 None
(100, 280, 3) (100, 280, 3)
9 None
(100, 280, 3) (100, 280, 3)
10 None
(100, 280, 3) (100, 280, 3)
11 None
(100, 280, 3) (100, 280, 3)
12 None
(100, 280, 3) (100, 280, 3)
13 None
(100, 280, 3) (100, 280, 3)
14 None
(100, 280, 3) (100, 280, 3)
15 None
(100, 280, 3) (100, 280, 3)
16 None
(100, 280, 3) (100, 280, 3)
17 None
(100, 280, 3) (100, 280, 3)
18 None
(100, 280, 3) (100, 280, 3)
19 None
(100, 280, 3) (100, 280, 3)
20 None
(100, 280, 3) (100, 280, 3)
21 None
(100, 280, 3) (100, 280, 3)
22 None
(100, 280, 3) (100, 280, 3)
23 None
(100, 280, 3) (100, 280, 3)
24 None
(100, 280, 3) (100, 280, 3)
25 None
(100, 280, 3) (100, 280, 3)
26 None
(100, 280, 3) (100, 280, 3)
27

233 49
(100, 280, 3) (100, 280, 3)
234 50
(100, 280, 3) (100, 280, 3)
235 50
(100, 280, 3) (100, 280, 3)
236 50
(100, 280, 3) (100, 280, 3)
237 50
(100, 280, 3) (100, 280, 3)
238 50
(100, 280, 3) (100, 280, 3)
239 50
(100, 280, 3) (100, 280, 3)
240 50
(100, 280, 3) (100, 280, 3)
241 50
(100, 280, 3) (100, 280, 3)
242 50
(100, 280, 3) (100, 280, 3)
243 50
(100, 280, 3) (100, 280, 3)
244 50
(100, 280, 3) (100, 280, 3)
245 50
(100, 280, 3) (100, 280, 3)
246 50
(100, 280, 3) (100, 280, 3)
247 50
(100, 280, 3) (100, 280, 3)
248 50
(100, 280, 3) (100, 280, 3)
249 50
(100, 280, 3) (100, 280, 3)
250 50
(100, 280, 3) (100, 280, 3)
251 50
(100, 280, 3) (100, 280, 3)
252 50
(100, 280, 3) (100, 280, 3)
253 50
(100, 280, 3) (100, 280, 3)
254 50
(100, 280, 3) (100, 280, 3)
255 50
(100, 280, 3) (100, 280, 3)
256 50
(100, 280, 3) (100, 280, 3)
257 50
(100, 280, 3) (100, 280, 3)
258 50
(100, 280, 3) (100, 280, 3)
259 50
(100, 280, 3) (100, 280, 3)
260 50
(100, 280, 3) (100, 280, 3)
261 None
(100, 280, 

460 None
(100, 280, 3) (100, 280, 3)
461 None
(100, 280, 3) (100, 280, 3)
462 None
(100, 280, 3) (100, 280, 3)
463 None
(100, 280, 3) (100, 280, 3)
464 None
(100, 280, 3) (100, 280, 3)
465 None
(100, 280, 3) (100, 280, 3)
466 None
(100, 280, 3) (100, 280, 3)
467 None
(100, 280, 3) (100, 280, 3)
468 None
(100, 280, 3) (100, 280, 3)
469 None
(100, 280, 3) (100, 280, 3)
470 None
(100, 280, 3) (100, 280, 3)
471 None
(100, 280, 3) (100, 280, 3)
472 105
(100, 280, 3) (100, 280, 3)
473 105
(100, 280, 3) (100, 280, 3)
474 105
(100, 280, 3) (100, 280, 3)
475 105
(100, 280, 3) (100, 280, 3)
476 105
(100, 280, 3) (100, 280, 3)
477 105
(100, 280, 3) (100, 280, 3)
478 105
(100, 280, 3) (100, 280, 3)
479 105
(100, 280, 3) (100, 280, 3)
480 105
(100, 280, 3) (100, 280, 3)
481 105
(100, 280, 3) (100, 280, 3)
482 105
(100, 280, 3) (100, 280, 3)
483 105
(100, 280, 3) (100, 280, 3)
484 105
(100, 280, 3) (100, 280, 3)
485 105
(100, 280, 3) (100, 280, 3)
486 105
(100, 280, 3) (100, 280, 3)
487 105
(100, 28

687 112
(100, 280, 3) (100, 280, 3)
688 112
(100, 280, 3) (100, 280, 3)
689 112
(100, 280, 3) (100, 280, 3)
690 112
(100, 280, 3) (100, 280, 3)
691 112
(100, 280, 3) (100, 280, 3)
692 112
(100, 280, 3) (100, 280, 3)
693 112
(100, 280, 3) (100, 280, 3)
694 112
(100, 280, 3) (100, 280, 3)
695 112
(100, 280, 3) (100, 280, 3)
696 112
(100, 280, 3) (100, 280, 3)
697 112
(100, 280, 3) (100, 280, 3)
698 112
(100, 280, 3) (100, 280, 3)
699 112
(100, 280, 3) (100, 280, 3)
700 112
(100, 280, 3) (100, 280, 3)
701 112
(100, 280, 3) (100, 280, 3)
702 112
(100, 280, 3) (100, 280, 3)
703 112
(100, 280, 3) (100, 280, 3)
704 112
(100, 280, 3) (100, 280, 3)
705 112
(100, 280, 3) (100, 280, 3)
706 112
(100, 280, 3) (100, 280, 3)
707 112
(100, 280, 3) (100, 280, 3)
708 112
(100, 280, 3) (100, 280, 3)
709 112
(100, 280, 3) (100, 280, 3)
710 112
(100, 280, 3) (100, 280, 3)
711 112
(100, 280, 3) (100, 280, 3)
712 112
(100, 280, 3) (100, 280, 3)
713 112
(100, 280, 3) (100, 280, 3)
714 None
(100, 280, 3) (100,

915 None
(100, 280, 3) (100, 280, 3)
916 None
(100, 280, 3) (100, 280, 3)
917 None
(100, 280, 3) (100, 280, 3)
918 None
(100, 280, 3) (100, 280, 3)
919 None
(100, 280, 3) (100, 280, 3)
920 None
(100, 280, 3) (100, 280, 3)
921 None
(100, 280, 3) (100, 280, 3)
922 None
(100, 280, 3) (100, 280, 3)
923 None
(100, 280, 3) (100, 280, 3)
924 None
(100, 280, 3) (100, 280, 3)
925 None
(100, 280, 3) (100, 280, 3)
926 None
(100, 280, 3) (100, 280, 3)
927 None
(100, 280, 3) (100, 280, 3)
928 None
(100, 280, 3) (100, 280, 3)
929 None
(100, 280, 3) (100, 280, 3)
930 None
(100, 280, 3) (100, 280, 3)
931 None
(100, 280, 3) (100, 280, 3)
932 None
(100, 280, 3) (100, 280, 3)
933 None
(100, 280, 3) (100, 280, 3)
934 None
(100, 280, 3) (100, 280, 3)
935 None
(100, 280, 3) (100, 280, 3)
936 None
(100, 280, 3) (100, 280, 3)
937 None
(100, 280, 3) (100, 280, 3)
938 None
(100, 280, 3) (100, 280, 3)
939 None
(100, 280, 3) (100, 280, 3)
940 None
(100, 280, 3) (100, 280, 3)
941 None
(100, 280, 3) (100, 280, 3)
9

KeyboardInterrupt: 